In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import tensorboard
from datetime import datetime
import os

2024-04-04 06:06:45.017844: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-04 06:06:45.019579: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 06:06:45.045559: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 06:06:45.045593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 06:06:45.046359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
base_path = os.getcwd()

In [3]:
dfs = []

base_path = os.getcwd()
for f in range(1,5):

    path = f'{base_path}/MachineLearningCVE/UNSW-NB15_{f}.csv'  # There are 4 input csv files
    dfs.append(pd.read_csv(path, header = None))
combined_data = pd.concat(dfs).reset_index(drop=True)  # Concat all to a single df

/tmp/ipykernel_459118/927831727.py:7: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path, header = None))
/tmp/ipykernel_459118/927831727.py:7: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path, header = None))


In [4]:
combined_data.columns

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48],
      dtype='int64')

In [5]:
dataset_columns = pd.read_csv(f'{base_path}/MachineLearningCVE/NUSW-NB15_features.csv',encoding='ISO-8859-1')
combined_data.columns = dataset_columns['Name']
dataset_columns.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No.          49 non-null     int64 
 1   Name         49 non-null     object
 2   Type         49 non-null     object
 3   Description  49 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.7+ KB


In [6]:
# Setting Columns indexes for our Dataset Dataframe object.
combined_data.info()
combined_data.head(4)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540047 entries, 0 to 2540046
Data columns (total 49 columns):
 #   Column            Dtype  
---  ------            -----  
 0   srcip             object 
 1   sport             object 
 2   dstip             object 
 3   dsport            object 
 4   proto             object 
 5   state             object 
 6   dur               float64
 7   sbytes            int64  
 8   dbytes            int64  
 9   sttl              int64  
 10  dttl              int64  
 11  sloss             int64  
 12  dloss             int64  
 13  service           object 
 14  Sload             float64
 15  Dload             float64
 16  Spkts             int64  
 17  Dpkts             int64  
 18  swin              int64  
 19  dwin              int64  
 20  stcpb             int64  
 21  dtcpb             int64  
 22  smeansz           int64  
 23  dmeansz           int64  
 24  trans_depth       int64  
 25  res_bdy_len       int64  
 26  Sjit          

Name,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0


In [7]:
# combined_data['attack_cat'] = combined_data['attack_cat'].fillna(value='normal').apply(lambda x: x.strip().lower())
# combined_data['attack_cat'] = combined_data['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())
# combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
# combined_data['is_ftp_login'] = combined_data['is_ftp_login'].fillna(value=0)
# combined_data['is_ftp_login'] = np.where(combined_data['is_ftp_login']>1, 1, combined_data['is_ftp_login'])
# combined_data['service'] = combined_data['service'].apply(lambda x:"None" if x=='-' else x)
# combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
# combined_data['ct_flw_http_mthd'] = combined_data['ct_flw_http_mthd'].fillna(value=0)
# combined_data['service'] = combined_data['service'].apply(lambda x:"None" if x=='-' else x)
# combined_data[['service','ct_flw_http_mthd','is_ftp_login','ct_ftp_cmd','attack_cat','Label']]
# combined_data['ct_flw_http_mthd'] = combined_data['ct_flw_http_mthd'].fillna(value=0)
# combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
# combined_data.drop(columns=['srcip','sport','dstip','dsport','Label',],inplace=True)

In [8]:
combined_data.head()
combined_data.index.name = None
combined_data.reset_index(drop=True)

combined_data.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object', name='Name')

In [9]:
combined_data['attack_cat'] = combined_data['attack_cat'].fillna(value='normal').apply(lambda x: x.strip().lower())
combined_data['attack_cat'] = combined_data['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())
combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
combined_data['is_ftp_login'] = combined_data['is_ftp_login'].fillna(value=0)
combined_data['is_ftp_login'] = np.where(combined_data['is_ftp_login']>1, 1, combined_data['is_ftp_login'])
combined_data['service'] = combined_data['service'].apply(lambda x:"None" if x=='-' else x)
combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
combined_data['ct_flw_http_mthd'] = combined_data['ct_flw_http_mthd'].fillna(value=0)
combined_data['is_ftp_login'] = combined_data['is_ftp_login'].fillna(value=0)
combined_data['is_ftp_login'] = np.where(combined_data['is_ftp_login']>1, 1, combined_data['is_ftp_login'])
combined_data['service'] = combined_data['service'].apply(lambda x:"None" if x=='-' else x)
combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
combined_data['ct_flw_http_mthd'] = combined_data['ct_flw_http_mthd'].fillna(value=0)
combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
combined_data['is_ftp_login'] = combined_data['is_ftp_login'].fillna(value=0)
combined_data.drop(columns=['srcip','sport','dstip','dsport','Label',],inplace=True)












In [10]:
combined_data.head(20)
combined_data['attack_cat'].unique()

array(['normal', 'exploits', 'reconnaissance', 'dos', 'generic',
       'shellcode', 'fuzzers', 'worms', 'backdoor', 'analysis'],
      dtype=object)

In [69]:
# combined_data.to_csv(f'{base_path}/combined_data.csv')
'ccccc'
dataa = pd.read_csv(f"{base_path}/ccccccccc.csv",  chunksize= 100)
i = 1
total = 100
for chunk  in dataa:
    
    data = chunk.drop(columns=['Unnamed: 0'], axis=1)
    data.to_csv(f'{base_path}/ccccc/{i}.csv', index=False)
    data.__name__ = 'Name'
    print(data.columns)
    data = pd.read_csv(f'{base_path}/ccccc/{i}.csv')
    if i % total == 0 :
        break
    i+=1
    


Index(['proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat'],
      dtype='object')
Index(['proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login'

44

In [11]:
train, test = train_test_split(combined_data,test_size=0.2,random_state=16)
train, val = train_test_split(train,test_size=0.2,random_state=16)

In [12]:
x_train, y_train = train.drop(columns=['attack_cat']), train[['attack_cat']]
x_test, y_test = test.drop(columns=['attack_cat']), test[['attack_cat']]
x_val, y_val = val.drop(columns=['attack_cat']), val[['attack_cat']]



In [14]:
x_train.columns
len(['proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm'])

43

In [72]:
# type(x_train)
print(train.shape)
train


(1625629, 44)


Name,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat
1070804,tcp,FIN,0.438857,4776,3080,31,29,7,7,None,...,0.0,0,6,3,4,5,1,1,3,normal
1054569,tcp,CON,0.026163,2230,13900,31,29,7,10,None,...,0.0,0,2,7,2,1,1,1,1,normal
1548299,tcp,FIN,0.014954,2854,26584,31,29,7,16,None,...,0.0,0,9,6,4,8,1,1,1,normal
2305760,udp,INT,0.000009,114,0,254,0,0,0,dns,...,0.0,0,33,33,17,17,17,17,33,generic
1989356,udp,CON,0.001562,544,304,31,29,0,0,None,...,0.0,0,6,7,5,6,1,1,3,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358801,tcp,FIN,0.046476,320,1890,31,29,1,2,ftp-data,...,0.0,0,3,5,3,7,1,1,2,normal
556159,tcp,FIN,0.027755,5928,8010,31,29,14,17,ssh,...,0.0,0,1,1,8,3,1,1,1,normal
1408939,udp,INT,0.000003,264,0,60,0,0,0,dns,...,0.0,0,39,39,18,18,18,18,39,normal
1894282,tcp,FIN,0.057911,2766,27392,31,29,7,16,None,...,0.0,0,2,2,3,4,1,1,1,normal


In [73]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape #, x_val.shape, y_val.shape

((1625629, 43), (1625629, 1), (508010, 43), (508010, 1))

In [15]:

cat_col = ['proto', 'service', 'state']
num_col = list(set(x_train.columns) - set(cat_col))

In [16]:
x_train.shape , type(x_train)
num_col

['Djit',
 'ct_src_dport_ltm',
 'Ltime',
 'ct_flw_http_mthd',
 'Sload',
 'dttl',
 'is_ftp_login',
 'ackdat',
 'ct_dst_ltm',
 'tcprtt',
 'ct_dst_sport_ltm',
 'trans_depth',
 'sbytes',
 'is_sm_ips_ports',
 'smeansz',
 'ct_ftp_cmd',
 'Dpkts',
 'Sjit',
 'Dload',
 'ct_state_ttl',
 'Sintpkt',
 'ct_src_ ltm',
 'dmeansz',
 'swin',
 'dbytes',
 'ct_srv_src',
 'ct_srv_dst',
 'sttl',
 'Dintpkt',
 'Spkts',
 'dwin',
 'Stime',
 'ct_dst_src_ltm',
 'sloss',
 'dloss',
 'dur',
 'stcpb',
 'res_bdy_len',
 'synack',
 'dtcpb']

In [17]:
labels = train['attack_cat']




In [18]:
scaler = StandardScaler()
scaler = scaler.fit(x_train[num_col])
x_train[num_col] = scaler.transform(x_train[num_col])
x_test[num_col] = scaler.transform(x_test[num_col])
x_val[num_col] = scaler.transform(x_val[num_col])


In [19]:
x_train.shape

(1625629, 43)

In [78]:
type(x_train), x_train.shape

(pandas.core.frame.DataFrame, (1625629, 43))

In [20]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), cat_col)], remainder='passthrough')
x_train = np.array(ct.fit_transform(x_train))
x_test = np.array(ct.transform(x_test))
x_val = np.array(ct.transform(x_val))

In [22]:
attacks = y_train['attack_cat'].unique()
x_train.shape

(1625629, 204)

In [81]:
ct1 = ColumnTransformer(transformers=[('encoder', OneHotEncoder(categories=[attacks],sparse_output=False), ['attack_cat'])], remainder='passthrough')
y_train = np.array(ct1.fit_transform(y_train))
y_test = np.array(ct1.transform(y_test))
y_val = np.array(ct1.transform(y_val))

In [36]:
# x_train
import os
import pandas as pd 
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

base_path = os.getcwd()
dataset_columns = pd.read_csv(f'{base_path}/MachineLearningCVE/NUSW-NB15_features.csv',encoding='ISO-8859-1')
path = f'{base_path}/MachineLearningCVE/UNSW-NB15_1.csv'  # There are 4 input csv files
data = pd.read_csv(path, header = None)
data = data.reset_index(drop=True)
data.columns = dataset_columns['Name']

/tmp/ipykernel_459118/989986066.py:12: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path, header = None)


In [2]:
data.head(2)
# x_train, y_train = data.drop(columns=['attack_cat']), data[['attack_cat']]


In [37]:
def PreProcess(input_data):
    
    
    
    
    
    
    input_data['attack_cat'] = input_data['attack_cat'].fillna(value='normal').apply(lambda x: x.strip().lower())
    input_data['attack_cat'] = input_data['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())
    input_data['ct_ftp_cmd'] = input_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
    input_data['is_ftp_login'] = input_data['is_ftp_login'].fillna(value=0)
    input_data['is_ftp_login'] = np.where(input_data['is_ftp_login']>1, 1, input_data['is_ftp_login'])
    input_data['service'] = input_data['service'].apply(lambda x:"None" if x=='-' else x)
    input_data['ct_ftp_cmd'] = input_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
    input_data['ct_flw_http_mthd'] = input_data['ct_flw_http_mthd'].fillna(value=0)
    input_data['is_ftp_login'] = input_data['is_ftp_login'].fillna(value=0)
    input_data['is_ftp_login'] = np.where(input_data['is_ftp_login']>1, 1, input_data['is_ftp_login'])
    input_data['service'] = input_data['service'].apply(lambda x:"None" if x=='-' else x)
    input_data['ct_ftp_cmd'] = input_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
    input_data['ct_flw_http_mthd'] = input_data['ct_flw_http_mthd'].fillna(value=0)
    input_data['ct_ftp_cmd'] = input_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
    input_data['is_ftp_login'] = input_data['is_ftp_login'].fillna(value=0)
    input_data.drop(columns=['srcip','sport','dstip','dsport','Label',],inplace=True)
    
    return input_data

def processData(input_data):
    #--------------------------------
    base_path = os.getcwd()
    dfs = []
    for f in range(1,5):

        path = f'{base_path}/MachineLearningCVE/UNSW-NB15_{f}.csv'  # There are 4 input csv files
        dfs.append(pd.read_csv(path, header = None))
    combined_data = pd.concat(dfs).reset_index(drop=True)
    dataset_columns = pd.read_csv(f'{base_path}/MachineLearningCVE/NUSW-NB15_features.csv',encoding='ISO-8859-1')
    combined_data.columns = dataset_columns['Name']
    combined_data = PreProcess(input_data= combined_data)
    
    x_train, y_train = combined_data.drop(columns=['attack_cat']), combined_data[['attack_cat']]
    
    print("\n############", x_train.columns, "\n##############")
    cat_col = ['proto', 'service', 'state']
    num_col = list(set(x_train.columns) - set(cat_col))
    
    
    
    scaler = StandardScaler()
    scaler = scaler.fit(x_train[num_col])
    # combined_data[num_col] = scaler.transform(combined_data[num_col])
    # x_test[num_col] = scaler.transform(x_test[num_col])
    # x_val[num_col] = scaler.transform(x_val[num_col])
    
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), cat_col)], remainder='passthrough')
    x_train = np.array(ct.fit_transform(x_train))
    print(x_train.shape,"\n***********")
    # x_test = np.array(ct.transform(x_test))
    # x_val = np.array(ct.transform(x_val))
    # ------------------------------------
    input_data = PreProcess(input_data=input_data)
    x_return, y_train = input_data.drop(columns=['attack_cat']), input_data[['attack_cat']]
    print("\n############", x_return.columns, "\n##############")
  
    
    x_return[num_col] = scaler.transform(input_data[num_col])
    x_return = np.array(ct.transform(input_data))
    print(x_return.shape, "\n******************")
    del(scaler)
    del(ct)
    del(combined_data)
    return x_return

In [38]:
data_processed = processData(data)

/tmp/ipykernel_459118/2875094077.py:34: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path, header = None))
/tmp/ipykernel_459118/2875094077.py:34: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path, header = None))



############ Index(['proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm'],
      dtype='object', name='Name') 
##############
(2540047, 204) 
***********

############ Index(['proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_s

In [ ]:
len(['proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm'])

43

In [39]:
type(data_processed)
# np.reshape(data_processed.shape, newshape=((data_processed.shape[0]), 205,1))
data_processed.shape

(700001, 204)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, LSTM, Dense, Dropout

# Define model parameters
n_features = x_train.shape[1]  # Number of features from training data

# Model Definition
model = tf.keras.Sequential([
    # First convolutional block
    Conv1D(filters=16, kernel_size=1, activation='relu', input_shape=(n_features,1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    # First LSTM block
    LSTM(units=16, return_sequences=True),

    # Second convolutional block
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    # Second LSTM block
    LSTM(units=32, return_sequences=True),

    # Third convolutional block
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    # Third LSTM block
    LSTM(units=64),

    # Dense layers
    Dense(64, activation='relu'),
    Dropout(0.2),  # Adjusted dropout rate for dense layer
    Dense(10, activation='softmax')
])


In [22]:
logdir=f"{base_path}/logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

callbacks = [
tf.keras.callbacks.ModelCheckpoint(f"{base_path}/IDS_cnn_lstm.keras",
 save_best_only=True,
    monitor="val_loss",
    ),
tensorboard_callback

]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 204, 16)           32        
                                                                 


 max_pooling1d (MaxPooling1  (None, 102, 16)           0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 102, 16)           64        
 Normalization)                                                  
                                                                 
 lstm (LSTM)                 (None, 102, 16)           2112      
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 32)           1568      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 50, 32)            0         
 g1D)                                                            
                                                                 
 batch_normalization_1 (Bat  (None, 50, 32)            128       
 chNormalization)                                                
          

In [40]:
import tensorflow as tf
checkpoint_path = f'{base_path}/IDS_cnn_lstm.keras'


In [41]:
lstm_cnn  = tf.keras.models.load_model(filepath=checkpoint_path)

In [42]:
tf.__version__
x_train.shape
x_test.shape


(508010, 204)

In [45]:
# lstm_cnn.evaluate(x_test, y_test)

prediction = lstm_cnn.predict(data_processed)


21876/21876 [==============================] - 358s 16ms/step


In [49]:
print(np.argmax(prediction,  axis=1))

[1 4 1 ... 0 4 3]


In [27]:
predicttt = x_test[97300].reshape(-1,204,1)
prediction = lstm_cnn(predicttt)
predicttt.shape



(1, 204, 1)

In [28]:
x_test.shape

prediction= np.reshape(prediction,newshape= (10,))
print(prediction)
print(np.argmax(prediction))
prediction[np.argmax(prediction)]

[1.0000000e+00 1.4005308e-11 8.9681290e-13 5.6154154e-11 1.5691245e-11
 7.2458668e-14 6.1987902e-19 1.7581118e-21 4.0863531e-32 3.6948451e-37]
0


1.0

In [29]:
x_test[400]
y_test[400]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [30]:
def predict (x):
    predictt = x.reshape(-1,204,1)
    predictionnn = lstm_cnn(predictt)
    # predictionnn=   np.reshape(prediction,newshape= (10,), )
    print(predictionnn[0])
    key = np.argmax(predictionnn[0])
    print(predictionnn[0][key])
    return key

In [31]:
print(predict(x_train[97300]))
print(np.argmax(y_train[97300]))


tf.Tensor(
[9.6336205e-04 1.8636679e-02 3.5754174e-03 7.7781832e-01 6.6556618e-02
 9.2024595e-02 4.3868893e-03 1.6119589e-05 2.7200003e-05 3.5994764e-02], shape=(10,), dtype=float32)
tf.Tensor(0.7778183, shape=(), dtype=float32)
3
9


In [32]:
array = np.reshape([[9,30,7], [1,12222,1]], (6,))

In [33]:
np.argmax(x_train, )
x_train[162562]
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [34]:
type(x_test)

numpy.ndarray

In [47]:

for index, row in combined_data.iterrows():
    for key in row.keys():
        print(key)
        print('\n')
        print(combined_data[key].unique())
        print("\n ------------------------------------------------------------------ \n")
    break




proto


['udp' 'arp' 'tcp' 'ospf' 'icmp' 'igmp' 'sctp' 'udt' 'sep' 'sun-nd'
 'swipe' 'mobile' 'pim' 'rtp' 'ipnip' 'ip' 'ggp' 'st2' 'egp' 'cbt' 'emcon'
 'nvp' 'igp' 'xnet' 'argus' 'bbn-rcc' 'chaos' 'pup' 'hmp' 'mux' 'dcn'
 'prm' 'trunk-1' 'xns-idp' 'trunk-2' 'leaf-1' 'leaf-2' 'irtp' 'rdp'
 'iso-tp4' 'netblt' 'mfe-nsp' 'merit-inp' '3pc' 'xtp' 'idpr' 'tp++' 'ddp'
 'idpr-cmtp' 'ipv6' 'il' 'idrp' 'ipv6-frag' 'sdrp' 'ipv6-route' 'gre'
 'rsvp' 'mhrp' 'bna' 'esp' 'i-nlsp' 'narp' 'ipv6-no' 'tlsp' 'skip'
 'ipv6-opts' 'any' 'cftp' 'sat-expak' 'kryptolan' 'rvd' 'ippc' 'sat-mon'
 'ipcv' 'visa' 'cpnx' 'cphb' 'wsn' 'pvp' 'br-sat-mon' 'wb-mon' 'wb-expak'
 'iso-ip' 'secure-vmtp' 'vmtp' 'vines' 'ttp' 'nsfnet-igp' 'dgp' 'tcf'
 'eigrp' 'sprite-rpc' 'larp' 'mtp' 'ax.25' 'ipip' 'micp' 'aes-sp3-d'
 'encap' 'etherip' 'pri-enc' 'gmtp' 'pnni' 'ifmp' 'aris' 'qnx' 'a/n'
 'scps' 'snp' 'ipcomp' 'compaq-peer' 'ipx-n-ip' 'vrrp' 'zero' 'pgm' 'iatp'
 'ddx' 'l2tp' 'srp' 'stp' 'smp' 'uti' 'sm' 'ptp' 'fire' 'crtp' 'isis'


In [44]:
{
    "proto": "", # ['udp' 'arp' 'tcp' 'ospf' 'icmp' 'igmp' 'sctp' 'udt' 'sep' 'sun-nd' 'swipe' 'mobile' 'pim' 'rtp' 'ipnip' 'ip' 'ggp' 'st2' 'egp' 'cbt' 'emcon' 'nvp' 'igp' 'xnet' 'argus' 'bbn-rcc' 'chaos' 'pup' 'hmp' 'mux' 'dcn' 'prm' 'trunk-1' 'xns-idp' 'trunk-2' 'leaf-1' 'leaf-2' 'irtp' 'rdp' 'iso-tp4' 'netblt' 'mfe-nsp' 'merit-inp' '3pc' 'xtp' 'idpr' 'tp++' 'ddp' 'idpr-cmtp' 'ipv6' 'il' 'idrp' 'ipv6-frag' 'sdrp' 'ipv6-route' 'gre' 'rsvp' 'mhrp' 'bna' 'esp' 'i-nlsp' 'narp' 'ipv6-no' 'tlsp' 'skip' 'ipv6-opts' 'any' 'cftp' 'sat-expak' 'kryptolan' 'rvd' 'ippc' 'sat-mon' 'ipcv' 'visa' 'cpnx' 'cphb' 'wsn' 'pvp' 'br-sat-mon' 'wb-mon' 'wb-expak' 'iso-ip' 'secure-vmtp' 'vmtp' 'vines' 'ttp' 'nsfnet-igp' 'dgp' 'tcf' 'eigrp' 'sprite-rpc' 'larp' 'mtp' 'ax.25' 'ipip' 'micp' 'aes-sp3-d' 'encap' 'etherip' 'pri-enc' 'gmtp' 'pnni' 'ifmp' 'aris' 'qnx' 'a/n' 'scps' 'snp' 'ipcomp' 'compaq-peer' 'ipx-n-ip' 'vrrp' 'zero' 'pgm' 'iatp' 'ddx' 'l2tp' 'srp' 'stp' 'smp' 'uti' 'sm' 'ptp' 'fire' 'crtp' 'isis' 'crudp' 'sccopmce' 'sps' 'pipe' 'iplt' 'unas' 'fc' 'ib']

    "state": "", # ['CON' 'INT' 'FIN' 'URH' 'REQ' 'ECO' 'RST' 'CLO' 'TXD' 'URN' 'no' 'ACC' 'PAR' 'MAS' 'TST' 'ECR']
    "dur": "", # 1.055000e-03 3.613300e-02 1.119000e-03 ... 1.616038e+00 2.200934e+00 5.649980e-01
    "sbytes": "" ,      # [   132    528    146 ...  86346  35110 269883]
    "dbytes": "",       #[   164    304    178 ...  60216 166054 772406]
    "sttl": "",         #[ 31   0 254  62   1  64 255  60  29 252  32  30  63]
    "dttl": "",         #[ 29   0 252  60  31  30  62  32 254 253  64]
    "sloss": "",        # int
    "dloss": "",        #int
    "service": "",          # ['dns' 'None' 'http' 'smtp' 'ftp-data' 'ftp' 'ssh' 'pop3' 'snmp' 'ssl' 'irc' 'radius' 'dhcp']
    "Sload": "",        #  [500473.9375   87676.08594 521894.5313  ...  12496.51367 198981.25 24465.67188]
    "Dload": "",        # [  621800.9375     50480.17188   636282.375   ...   598375.0625    10915975.        146610.7656 ]
    "Spkts": "",        # int
    "Dpkts": "",        #  int 
    "swin": "",         # int 
    "dwin": "",         # int 
    "stcpb": "",        # large unsigned int
    "dtcpb": "",        # large unsigned int
    "smeansz": "",          # int  [  66  132   73 ... 1440 1368 1485]
    "dmeansz": "",          # int  [  82   76   89 ... 1391 1049 1389]
    "trans_depth": "",          #  [  0   1   2   3   4   8 131   9  80 155   5 163 172  39]
    "res_bdy_len": "",          # [     0  12026     72 ...  28698  79745 372266]
    "Sjit": "",         #  [   0.          9.89101    29.682221 ... 2868.893137  116.321921 1221.136547]
    "Djit": "",         # [   0.         10.682733   34.37034  ... 1951.035944   69.874077  30.096029]
    "Stime": "",        #  [1421927414 1421927415 1421927416 ... 1424262066 1424262067 1424262068]
    "Ltime": "",        #  [1421927414 1421927415 1421927416 ... 1424262067 1424262068 1424262069]
    "Sintpkt": "",          #  [1.70000e-02 7.00500e+00 4.30000e-02 ... 3.86097e+01 2.12313e+00
    "Dintpkt": "",          # [1.3000000e-02 7.5643330e+00 1.4000000e-02 ... 7.2915602e+01 3.7571442e+01
    "tcprtt": "",       #  [0.       0.051475 0.066088 ... 0.164316 0.190234 0.139446]
    "synack": "",       #  [0.       0.006528 0.017959 ... 0.057804 0.094404 0.053884]
    "ackdat": "",       #  [0.       0.044947 0.048129 ... 0.093514 0.085562 0.093535]
    "is_sm_ips_ports": "",          # [0, 1]
    "ct_state_ttl": "",         #  [0 2 1 3 4 6 5]
    "ct_flw_http_mthd": "",         #  [ 0.  1.  2.  4. 14.  8.  6. 12. 10.  3.  5. 36.  9. 16. 25. 30.]
    "is_ftp_login": "",         #   [0. 1.]
    "ct_ftp_cmd": "",       #  [0 1 6 2 4 8 5 3]
    "ct_srv_src": "",       #  [ 3  2 12  6  7  8  5  1 10  9  4 13 14 18 11 20 16 17 24 15 21 30 19 22 29 35 27 31 39 23 26 25 28 34 38 32 41 44 33 37 36 42 40 43 63 60 46 48 49 53 52 50 51 59 47 55 54 45 58 57 56 66 61 64 67]
    "ct_srv_dst": "",       # [ 7  4  8  9  2  1  6  3 13  5 12 10 16 11 17 23 15 14 18 21 29 19 30 20 35 27 31 39 26 25 22 24 34 36 33 32 28 40 42 38 62 60 37 46 48 43 41 44 49 53 52 50 51 59 47 55 54 45 58 57 56 66 61 64 67 63]
    "ct_dst_ltm": "",       #  [ 1  2  3  5  4  9  8 11 16  7 10  6 12 18 19 13 15 25 14 17 22 32 23 24  26 34 33 21 20 28 27 31 39 30 35 42 29 38 36 40 37 60 46 48 49 53 52 41 47 44 45 54 43 55 59 58 56 51 50 61 57 67]
    "ct_src_ ltm": "",          #  [ 3  2  1  5  4  6  7  8  9 10 11 13 18 14 12 20 16 17 15 21 25 23 22 19 26 24 28 31 33 29 30 34 32 35 27 38 36 40 42 37 39 50 43 44 41 60 46 48 49 53 52 47 45 54 55 58 56 51 59 61 57 67]
    "ct_src_dport_ltm": "",         # [ 1  2  3  5  4  6  8  7 10 11  9 18 13 12 19 15 14 17 16 21 25 20 22 24  30 23 27 35 36 34 26 28 29 33 60 37 46 40 48 32 31 49 39 53 52 41 38 42 47 44 45 54 43 55 59 58 56 51 50 61 57 67]
    "ct_dst_sport_ltm": "",         # [ 1  2  4  3  7  5  6  8 14 12 21  9 11 10 15 20 17 13 19 25 18 23 16 31 27 34 30 26 29 33 22 24 60 37 46 40 48 49 39 53 52 47 28 32 36 35 38 54 51 58 41 57 56 50 44]
    "ct_dst_src_ltm": "",       # [ 1  2  4  3  5  6  7  9 10 11  8 12 14 15 13 16 21 25 19 17 22 18 20 23 24 29 27 38 31 34 30 33 26 40 63 60 37 39 46 28 36 48 32 35 43 42 41 44 49 53 52 50 51 59 47 55 54 45 58 57 56 66 61 65 67]
    "attack_cat": "",       #  ['normal' 'exploits' 'reconnaissance' 'dos' 'generic' 'shellcode' 'fuzzers' 'worms' 'backdoor' 'analysis']
}

{'proto': '',
 'state': '',
 'dur': '',
 'sbytes': '',
 'dbytes': '',
 'sttl': '',
 'dttl': '',
 'sloss': '',
 'dloss': '',
 'service': '',
 'Sload': '',
 'Dload': '',
 'Spkts': '',
 'Dpkts': '',
 'swin': '',
 'dwin': '',
 'stcpb': '',
 'dtcpb': '',
 'smeansz': '',
 'dmeansz': '',
 'trans_depth': '',
 'res_bdy_len': '',
 'Sjit': '',
 'Djit': '',
 'Stime': '',
 'Ltime': '',
 'Sintpkt': '',
 'Dintpkt': '',
 'tcprtt': '',
 'synack': '',
 'ackdat': '',
 'is_sm_ips_ports': '',
 'ct_state_ttl': '',
 'ct_flw_http_mthd': '',
 'is_ftp_login': '',
 'ct_ftp_cmd': '',
 'ct_srv_src': '',
 'ct_srv_dst': '',
 'ct_dst_ltm': '',
 'ct_src_ ltm': '',
 'ct_src_dport_ltm': '',
 'ct_dst_sport_ltm': '',
 'ct_dst_src_ltm': '',
 'attack_cat': ''}

In [214]:

import pickle5 as pickle 

with open(f"{base_path}/ct.pickle", "wb") as handle:
    pickle.dump(ct, handle)
    
    
ct1, scaler

with open(f"{base_path}/ct1.pickle", "wb") as handle:
    pickle.dump(ct1, handle)
    
with open(f"{base_path}/scalar.pickle", "wb") as handle:
    pickle.dump(scaler, handle)

: 

## Code For the APi

In [67]:
import os 
import pickle5 as pickle 

base_path = os.getcwd()


with open(f"{base_path}/ct.pickle", "rb") as handle:
    ct = pickle.load(handle)
    
    

with open(f"{base_path}/ct1.pickle", "rb") as handle:
    ct1 =  pickle.load(handle)
    
with open(f"{base_path}/scalar.pickle", "rb") as handle:
    scaler = pickle.load(handle)

In [68]:
def pipeline (sample_dataframe):
    sample_dataframe['attack_cat'] = sample_dataframe['attack_cat'].fillna(value='normal').apply(lambda x: x.strip().lower())
    sample_dataframe['attack_cat'] = sample_dataframe['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())
    sample_dataframe['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
    sample_dataframe['is_ftp_login'] = sample_dataframe['is_ftp_login'].fillna(value=0)
    sample_dataframe['is_ftp_login'] = np.where(sample_dataframe['is_ftp_login']>1, 1, sample_dataframe['is_ftp_login'])
    sample_dataframe['service'] = sample_dataframe['service'].apply(lambda x:"None" if x=='-' else x)
    sample_dataframe['ct_ftp_cmd'] = sample_dataframe['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
    sample_dataframe['ct_flw_http_mthd'] = sample_dataframe['ct_flw_http_mthd'].fillna(value=0)
    sample_dataframe['is_ftp_login'] = sample_dataframe['is_ftp_login'].fillna(value=0)
    sample_dataframe['is_ftp_login'] = np.where(sample_dataframe['is_ftp_login']>1, 1, sample_dataframe['is_ftp_login'])
    sample_dataframe['service'] = combined_data['service'].apply(lambda x:"None" if x=='-' else x)
    sample_dataframe['ct_ftp_cmd'] = sample_dataframe['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
    sample_dataframe['ct_flw_http_mthd'] = sample_dataframe['ct_flw_http_mthd'].fillna(value=0)
    sample_dataframe['ct_ftp_cmd'] = sample_dataframe['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
    sample_dataframe['is_ftp_login'] = sample_dataframe['is_ftp_login'].fillna(value=0)
    return sample_dataframe


In [86]:
import pickle5 as pickle



def staderdized (sample_dataframe):
    base_path = os.getcwd()
    
    with open(f"{base_path}/ct.pickle", "rb") as handle:
        ct = pickle.load(handle)
        
        

    with open(f"{base_path}/ct1.pickle", "rb") as handle:
        ct1 =  pickle.load(handle)
        
    with open(f"{base_path}/scalar.pickle", "rb") as handle:
        scaler = pickle.load(handle)
        
        
    sample_dataframe = pipeline(sample_dataframe=sample_dataframe)
    print(sample_dataframe.columns)
   
    
    
    x,y  =   sample_dataframe.drop(columns=['attack_cat']), sample_dataframe[['attack_cat']]
    cat_col = ['proto', 'service', 'state']
    num_col = list(set(x.columns) - set(cat_col))

    scaler = scaler.fit(x[num_col])
    x[num_col] = scaler.transform(x[num_col])
    x= np.array(ct.fit_transform(x))
    y = np.array(ct1.fit_transform(y))
    
    return x, y
   

In [90]:
sample_dataframe = pd.read_csv(f"{base_path}/ccccc/1.csv")
sample_dataframe.columns
x,y = staderdized(sample_dataframe=sample_dataframe)
lstm_cnn.predict(x)

Index(['proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat'],
      dtype='object')


ValueError: in user code:

    File "/home/workbook/ml/research_work/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/home/workbook/ml/research_work/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/workbook/ml/research_work/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/home/workbook/ml/research_work/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/home/workbook/ml/research_work/venv/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/workbook/ml/resea

In [19]:
cat_col = ['proto', 'service', 'state']
num_col = list(set(combined_data.columns) - set(cat_col))
cat_col, num_col

['proto', 'service', 'state'] # cat_col
['ct_dst_ltm','ct_flw_http_mthd','Sintpkt','dloss','trans_depth','ct_srv_src','Stime','ct_state_ttl','ct_src_ ltm','Dload','dwin','ct_dst_sport_ltm','sloss','is_sm_ips_ports','dtcpb','Spkts','smeansz','Ltime','sbytes','ct_srv_dst','Djit','Sjit','Sload','Dintpkt','tcprtt','synack','dttl','ackdat','is_ftp_login','ct_ftp_cmd','ct_src_dport_ltm','swin','stcpb','dbytes','Dpkts','sttl','dmeansz','res_bdy_len','dur','ct_dst_src_ltm']

NameError: name 'combined_data' is not defined

In [15]:
import pandas as pd
import os 

base_path = os.getcwd()
data = pd.read_csv(f"{base_path}/combined_data.csv", chunksize= 100)

In [17]:
i = 1
total = 100
for chunk  in data:
    chunk.to_csv(f'{base_path}/chunked/{i}.csv')
    
    if i % total == 0 :
        break
    i+=1